In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
from time import sleep
import pickle
import sys

In [2]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [3]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [4]:
main_data = all_groups.copy()

In [5]:
fnames = ['bm_feats_upd.pickle',#'bm_feats.pickle', 
          'un_bm_feats_upd.pickle',#'un_bm_feats.pickle', 
          'idf_1_2_upd.pickle', 
          'tf_1_2_upd.pickle',
          'tf_123_rupd.pickle',
          'idf_123_rupd.pickle',
          'idf_char_8_15_rupd.pickle', 
          'un_idf_123_rupd.pickle',
          'un_idf_1_2_upd.pickle',
          'un_tf_123_rupd.pickle',
          'un_tf_1_2_rupd.pickle',
          'basic_tf_idf_title_feats.pickle',
          'title_tfidf_vectorizer_1.pickle',
          'use-multilingual_3_unnormed_titles.pickle',
          'use-multilingual-large_3_unnormed_titles.pickle',
          'neuro_feats_unnormed_512_texts-multilang-qa-3_upd.pickle',
          'dbn_feats_advanced.pickle',#####
          'casc_feats_advanced.pickle',
          'click_feats_advanced.pickle',
          'click_feats_advanced1.pickle',
          'click_feats_advanced2.pickle',
          'click_feats_advanced3.pickle',
          'click_feats_advanced4.pickle',
          'click_feats_advanced5.pickle',
          'click_feats_advanced6.pickle',
          'click_feats_advanced7.pickle',
          'dbn_feats_pair.pickle',
          'casc_feats_pair.pickle',
          'click_feats_pair.pickle',
          'click_feats_pair1.pickle',
          'click_feats_pair2.pickle',
          'click_feats_pair3.pickle',
          'click_feats_pair4.pickle',
          'click_feats_pair5.pickle',
          'click_feats_pair6.pickle',
          'click_feats_pair7.pickle',
          'click_feats_extra.pickle',
          'click_feats_extra1.pickle',
          'click_feats_pair_extra.pickle',
          'click_feats_pair_extra1.pickle',
          #'geo.pickle',
          #'geo1.pickle',
          #'Q_geo.pickle',
          'tf_vectorizer_ch_4_7.pickle',
          'tfidf_vectorizer_ch_4_7.pickle']
           #'titles_distiluse-base-multilingual-cased-v1_unnormed_titles.pickle', #- делает хуже
          #'global_doc_ctr.pickle']####
          #'use-multilingual_3_unnormed_512_texts.pickle'] - не дает особого улучшения, наблюдается легкое падение#,
          #'basic_bm25_title_feats.pickle' - скор портится]#, 'tf_char_8_15.pickle']
#'basic_tf_idf_title_feats.pickle' - скор падает, заголовки надо нормализовывать
#глобальные фичи по заголовкам приносят пользу, надо все же посчитать

In [6]:
fnames

['bm_feats_upd.pickle',
 'un_bm_feats_upd.pickle',
 'idf_1_2_upd.pickle',
 'tf_1_2_upd.pickle',
 'tf_123_rupd.pickle',
 'idf_123_rupd.pickle',
 'idf_char_8_15_rupd.pickle',
 'un_idf_123_rupd.pickle',
 'un_idf_1_2_upd.pickle',
 'un_tf_123_rupd.pickle',
 'un_tf_1_2_rupd.pickle',
 'basic_tf_idf_title_feats.pickle',
 'title_tfidf_vectorizer_1.pickle',
 'use-multilingual_3_unnormed_titles.pickle',
 'use-multilingual-large_3_unnormed_titles.pickle',
 'neuro_feats_unnormed_512_texts-multilang-qa-3_upd.pickle',
 'dbn_feats_advanced.pickle',
 'casc_feats_advanced.pickle',
 'click_feats_advanced.pickle',
 'click_feats_advanced1.pickle',
 'click_feats_advanced2.pickle',
 'click_feats_advanced3.pickle',
 'click_feats_advanced4.pickle',
 'click_feats_advanced5.pickle',
 'click_feats_advanced6.pickle',
 'click_feats_advanced7.pickle',
 'dbn_feats_pair.pickle',
 'casc_feats_pair.pickle',
 'click_feats_pair.pickle',
 'click_feats_pair1.pickle',
 'click_feats_pair2.pickle',
 'click_feats_pair3.pickle',

In [7]:
for name in tqdm(fnames):
    with open(name, 'rb') as file:
        cur_feats = pickle.load(file)
    for key in cur_feats:
        main_data[key] = cur_feats[key]

100%|██████████| 42/42 [02:14<00:00,  3.20s/it]


In [8]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_example = pd.read_csv('sample.csv')
a = set(df_example['QueryId'].unique())
b = set(df_marks[0].unique())

In [9]:
X_train = main_data[main_data.QueryId.isin(b)]
X_test = main_data[main_data.QueryId.isin(a)]

In [10]:
X_train

,QueryId,DocumentId,BM25Plus_all,BM25L_all,BM25Okapi_all,BM25Plus_500,BM25L_500,BM25Okapi_500,BM25Plus_1000,BM25L_1000,...,QH_clicks_out,QH_shows_out,QH_clicks_out_prob,QH_shows_out_prob,QH_higher_prob,QH_lower_prob,QH_num_higher_cl,QH_num_lower_cl,tf_vectorizer_ch_4_7,tfidf_vectorizer_ch_4_7
258,2,19489,4.853305,13.461781,4.187953,11.045726,0.000000,0.000000,10.037674,0.000000,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.111745,0.063661
259,2,30946,6.485275,21.478373,4.722985,11.045726,0.000000,0.000000,10.037674,0.000000,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.131047,0.078779
260,2,33190,7.327317,94.865288,5.940038,12.231718,1.465672,0.803639,11.868497,7.444236,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.142667,0.090396
261,2,37143,7.272160,36.641482,5.350974,13.785434,5.881465,2.450641,13.637049,6.846024,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.113739,0.052193
262,2,37469,7.323723,114.055519,7.127059,13.500121,3.032782,1.721798,12.584131,6.287518,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.140706,0.089941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605567,6305,569261,1.686882,36.132067,8.976063,2.192933,1.504242,4.276721,1.783638,3.115273,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.018180,0.007429
605568,6305,572086,2.678695,14.508098,11.618221,2.794356,9.369428,6.406924,2.521010,15.486075,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.109525,0.067766
605569,6305,573311,1.364439,7.686769,6.680817,2.196260,1.510884,4.114362,1.781458,2.913615,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.073425,0.020940
605570,6305,579920,3.351007,104.104492,13.548248,6.406196,199.021581,11.287404,5.118262,158.927977,...,0,40,0.0,0.273973,0.054795,0.054795,8,8,0.870267,0.697952


In [11]:
s_marks = df_marks.sort_values(by=[0,1]).reset_index().drop(columns=['index'])
train_groups = s_marks.groupby([0]).size().values
y_train = s_marks[2].values

In [12]:
X_train

,QueryId,DocumentId,BM25Plus_all,BM25L_all,BM25Okapi_all,BM25Plus_500,BM25L_500,BM25Okapi_500,BM25Plus_1000,BM25L_1000,...,QH_clicks_out,QH_shows_out,QH_clicks_out_prob,QH_shows_out_prob,QH_higher_prob,QH_lower_prob,QH_num_higher_cl,QH_num_lower_cl,tf_vectorizer_ch_4_7,tfidf_vectorizer_ch_4_7
258,2,19489,4.853305,13.461781,4.187953,11.045726,0.000000,0.000000,10.037674,0.000000,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.111745,0.063661
259,2,30946,6.485275,21.478373,4.722985,11.045726,0.000000,0.000000,10.037674,0.000000,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.131047,0.078779
260,2,33190,7.327317,94.865288,5.940038,12.231718,1.465672,0.803639,11.868497,7.444236,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.142667,0.090396
261,2,37143,7.272160,36.641482,5.350974,13.785434,5.881465,2.450641,13.637049,6.846024,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.113739,0.052193
262,2,37469,7.323723,114.055519,7.127059,13.500121,3.032782,1.721798,12.584131,6.287518,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.140706,0.089941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605567,6305,569261,1.686882,36.132067,8.976063,2.192933,1.504242,4.276721,1.783638,3.115273,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.018180,0.007429
605568,6305,572086,2.678695,14.508098,11.618221,2.794356,9.369428,6.406924,2.521010,15.486075,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.109525,0.067766
605569,6305,573311,1.364439,7.686769,6.680817,2.196260,1.510884,4.114362,1.781458,2.913615,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.073425,0.020940
605570,6305,579920,3.351007,104.104492,13.548248,6.406196,199.021581,11.287404,5.118262,158.927977,...,0,40,0.0,0.273973,0.054795,0.054795,8,8,0.870267,0.697952


In [13]:
qifs = X_train['QueryId'].values

In [14]:
X_train = X_train.loc[:, ~X_train.columns.isin(['QueryId','DocumentId'])]

In [15]:
X_train

,BM25Plus_all,BM25L_all,BM25Okapi_all,BM25Plus_500,BM25L_500,BM25Okapi_500,BM25Plus_1000,BM25L_1000,BM25Okapi_1000,un_BM25Plus_all,...,QH_clicks_out,QH_shows_out,QH_clicks_out_prob,QH_shows_out_prob,QH_higher_prob,QH_lower_prob,QH_num_higher_cl,QH_num_lower_cl,tf_vectorizer_ch_4_7,tfidf_vectorizer_ch_4_7
258,4.853305,13.461781,4.187953,11.045726,0.000000,0.000000,10.037674,0.000000,0.000000,8.103071,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.111745,0.063661
259,6.485275,21.478373,4.722985,11.045726,0.000000,0.000000,10.037674,0.000000,0.000000,8.151390,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.131047,0.078779
260,7.327317,94.865288,5.940038,12.231718,1.465672,0.803639,11.868497,7.444236,0.977057,10.144223,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.142667,0.090396
261,7.272160,36.641482,5.350974,13.785434,5.881465,2.450641,13.637049,6.846024,2.833189,9.281509,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.113739,0.052193
262,7.323723,114.055519,7.127059,13.500121,3.032782,1.721798,12.584131,6.287518,1.226381,9.994651,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.140706,0.089941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605567,1.686882,36.132067,8.976063,2.192933,1.504242,4.276721,1.783638,3.115273,4.833943,2.094524,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.018180,0.007429
605568,2.678695,14.508098,11.618221,2.794356,9.369428,6.406924,2.521010,15.486075,7.927059,3.665570,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.109525,0.067766
605569,1.364439,7.686769,6.680817,2.196260,1.510884,4.114362,1.781458,2.913615,4.714421,1.583163,...,0,0,0.0,0.000000,0.000000,0.000000,0,0,0.073425,0.020940
605570,3.351007,104.104492,13.548248,6.406196,199.021581,11.287404,5.118262,158.927977,11.171278,4.692240,...,0,40,0.0,0.273973,0.054795,0.054795,8,8,0.870267,0.697952


In [17]:
for i in X_train.columns:
    print(i)

BM25Plus_all
BM25L_all
BM25Okapi_all
BM25Plus_500
BM25L_500
BM25Okapi_500
BM25Plus_1000
BM25L_1000
BM25Okapi_1000
un_BM25Plus_all
un_BM25L_all
un_BM25Okapi_all
un_BM25Plus_500
un_BM25L_500
un_BM25Okapi_500
un_BM25Plus_1000
un_BM25L_1000
un_BM25Okapi_1000
tfidf_vectorizer_1
tfidf_vectorizer_2
tf_vectorizer_1
tf_vectorizer_2
tf_vectorizer_123
tfidf_vectorizer_123
tfidf_vectorizer_ch_8_15
un_tfidf_vectorizer_123
un_tfidf_vectorizer_1
un_tfidf_vectorizer_2
un_tf_vectorizer_123
un_tf_vectorizer_1
un_tf_vectorizer_2
title_loc_tfidf_vectorizer_1
title_loc_tfidf_vectorizer_2
title_loc_tfidf_vectorizer_3
title_loc_tfidf_vectorizer_234
title_loc_tfidf_vectorizer_12345
title_loc_tfidf_vectorizer_ch_1_7
title_loc_tfidf_vectorizer_ch_8_15
title_loc_tf_vectorizer_1
title_loc_tf_vectorizer_2
title_loc_tf_vectorizer_3
title_loc_tf_vectorizer_234
title_loc_tf_vectorizer_12345
title_loc_tf_vectorizer_ch_1_7
title_loc_tf_vectorizer_ch_8_15
title_tfidf_vectorizer_1
use-multilingual/3_unnormed_titles
use-m

In [16]:
X_train.to_csv("export_train.csv")

In [17]:
with open("export_y.pickle","wb") as file:
    pickle.dump(y_train, file)

In [18]:
with open("qifs.pickle", "wb") as file:
    pickle.dump(qifs, file)

In [20]:
y_train

array([1, 1, 1, ..., 1, 0, 4], dtype=int64)

In [20]:
X_test.to_csv("export_test.csv")

In [21]:
1+2

3